<a href="https://colab.research.google.com/github/sudeepbhagat11/Fake-News-Classifier-Bidirectional-LSTM-RNN/blob/main/Fake_News_Classifier_Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

In [18]:
df = pd.read_csv('/content/WELFake_Dataset.csv')

In [19]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [20]:
df.shape

(72134, 4)

In [21]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [22]:
df = df.dropna()

In [23]:
df.shape

(71537, 4)

In [25]:
X=df.drop('label',axis=1)
y = df['label']
X.shape

(71537, 3)

In [26]:
y.shape

(71537,)

In [27]:
## Check whether dataset is balanced or not
y.value_counts()


label
1    36509
0    35028
Name: count, dtype: int64

In [28]:
import tensorflow as tf

In [29]:
tf.__version__

'2.15.0'

In [30]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [31]:
## vocabulary size
voc_size=5000

In [32]:
messages=X.copy()

In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
messages.reset_index(inplace=True)

In [36]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
66537
66538
66539
66540
66541
66542
66543
66544
66545
66546
66547
66548
66549
66550
66551
66552
66553
66554
66555
66556
66557
66558
66559
66560
66561
66562
66563
66564
66565
66566
66567
66568
66569
66570
66571
66572
66573
66574
66575
66576
66577
66578
66579
66580
66581
66582
66583
66584
66585
66586
66587
66588
66589
66590
66591
66592
66593
66594
66595
66596
66597
66598
66599
66600
66601
66602
66603
66604
66605
66606
66607
66608
66609
66610
66611
66612
66613
66614
66615
66616
66617
66618
66619
66620
66621
66622
66623
66624
66625
66626
66627
66628
66629
66630
66631
66632
66633
66634
66635
66636
66637
66638
66639
66640
66641
66642
66643
66644
66645
66646
66647
66648
66649
66650
66651
66652
66653
66654
66655
66656
66657
66658
66659
66660
66661
66662
66663
66664
66665
66666
66667
66668
66669
66670
66671
66672
66673
66674
66675
66676
66677
66678
66679
66680
66681
66682
66683
66684
66685
66686
66687
66688
66689
66690
66691
66692
66693
66694
6

In [37]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [38]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4635, 2746, 671, 2223, 1714, 2608, 1734, 1171, 1216, 113, 363, 3075],
 [1329,
  1412,
  378,
  302,
  744,
  108,
  2608,
  2441,
  4596,
  4331,
  4537,
  1825,
  3295,
  3075],
 [2942, 1184, 4887, 3269, 3181, 3579, 4204, 405, 3726, 1711, 3463, 4518],
 [1242, 1981, 192, 4130, 1546, 276, 4556, 455, 1331, 3592, 2331],
 [3153, 4204, 3470, 2752, 685, 4880, 2211, 3041, 3470],
 [983, 1339, 993, 4248, 2977, 2039, 2079, 3491, 1820, 505, 393],
 [592, 3490, 3280, 4338, 1981, 4505, 3579, 3968, 4786, 3075],
 [3889,
  1795,
  2192,
  1574,
  4174,
  3477,
  3518,
  3528,
  1567,
  3889,
  4718,
  2418,
  195,
  743,
  3075],
 [4591, 4278, 1511, 4080, 3358, 2303, 4886, 4278],
 [373, 164, 1286, 4323, 4379, 887, 2837, 1363],
 [2834, 4816, 2452, 1408, 4873, 3619, 3865, 4198, 3865, 1867],
 [3941, 651, 4338, 3830, 2385, 1564, 747, 2301, 4365],
 [2415, 3910, 357, 651, 1497, 2765, 1151, 3215, 691, 2971, 4013],
 [2108, 2281, 194, 2017, 3911, 2251, 2233, 2298, 2545, 463],
 [3914, 350, 1143, 2193, 1782, 59

In [39]:
# Embedding Representation. Ensure to do on-hot-repr before embedding

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ...,  113,  363, 3075],
       [   0,    0,    0, ..., 1825, 3295, 3075],
       [   0,    0,    0, ..., 1711, 3463, 4518],
       ...,
       [   0,    0,    0, ..., 1332,  844, 1547],
       [   0,    0,    0, ..., 3558, 4567, 4651],
       [   0,    0,    0, ..., 4367, 3215, 2538]], dtype=int32)

In [40]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [41]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [43]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
1498/1498 [==============================] - 79s 48ms/step - loss: 0.2981 - accuracy: 0.8684 - val_loss: 0.2774 - val_accuracy: 0.8833
Epoch 2/10
1498/1498 [==============================] - 69s 46ms/step - loss: 0.2112 - accuracy: 0.9132 - val_loss: 0.2590 - val_accuracy: 0.8946
Epoch 3/10
1498/1498 [==============================] - 70s 47ms/step - loss: 0.1766 - accuracy: 0.9305 - val_loss: 0.2548 - val_accuracy: 0.8958
Epoch 4/10
1498/1498 [==============================] - 69s 46ms/step - loss: 0.1457 - accuracy: 0.9440 - val_loss: 0.2657 - val_accuracy: 0.8968
Epoch 5/10
1498/1498 [==============================] - 70s 47ms/step - loss: 0.1185 - accuracy: 0.9550 - val_loss: 0.3135 - val_accuracy: 0.8890
Epoch 6/10
1498/1498 [==============================] - 73s 49ms/step - loss: 0.0940 - accuracy: 0.9648 - val_loss: 0.3570 - val_accuracy: 0.8904
Epoch 7/10
1498/1498 [==============================] - 70s 47ms/step - loss: 0.0739 - accuracy: 0.9723 - val_loss: 0.3983 -

In [44]:
y_pred=model.predict(X_test)

738/738 [==============================] - 11s 14ms/step


In [45]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [46]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[10333,  1360],
       [ 1337, 10578]])

In [47]:
print(accuracy_score(y_test,y_pred))

0.8857590647238225
